# Getting Started with the kluster.ai Batch API

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kluster-ai/klusterai-cookbook/blob/main/examples/getting-started.ipynb)

Welcome to the Batch API Getting Started Notebook!

<a href="https://kluster.ai/" target="_blank">kluster.ai</a> is a high-performance compute platform designed to make large-scale AI workloads accessible, efficient, and affordable. Our Batch API enables a variety of use cases, including summarization, classification, translation, and more, all without the need to manage infrastructure.

This notebook is designed to help you get started quickly with the <a href="https://kluster.ai/" target="_blank">kluster.ai</a> Batch API. It walks you through the essential code snippets from the <a href="https://docs.kluster.ai/get-started/api/" target="_blank">Getting Started documentation</a>, all in one place.

By running this notebook, you’ll:
- Learn how to use the API.
- Submit a simple batch job for LLM inference.
- Understand how to handle and interpret the API’s responses.

Each section corresponds to a step in the Getting Started guide, so you can follow along seamlessly.

## Setup

This step ensures that the openai Python library is installed or updated to the required version. This library will serve as the client for interacting with the kluster.ai Batch API.

In [ ]:
pip install "openai>=1.0.0"

## Creating Batch Jobs as JSONL Files

This step defines a batch of requests for the API to process. Each request includes a unique identifier (`custom_id`), the HTTP method (`POST`), the chat completions endpoint (`/v1/chat/completions`) and a body field that contains the request you want to send to the chat completions endpoint toghether with the `model` to be used and the conversational context ("messages"). These tasks are saved as a JSON Lines (`.jsonl`) file for efficient handling of multiple requests in a single upload.

You'll have to enter your personal kluster.ai API key (make sure it has no blank spaces). Remember to create a key in <a href="https://platform.kluster.ai/apikeys" target="_blank">platform.kluster.ai</a>, if you don't have one yet.

In [ ]:
from openai import OpenAI
import json
client = OpenAI(
    base_url="https://api.kluster.ai/v1",  
    api_key="INSERT_API_KEY", # Replace with your actual API key
)

tasks = [{
        "custom_id": "request-1",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "klusterai/Meta-Llama-3.1-8B-Instruct-Turbo",
            "messages": [
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": "What is the capital of Argentina?"},
            ],
            "max_tokens": 1000,
        },
    },
    {
        "custom_id": "request-2",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "klusterai/Meta-Llama-3.1-70B-Instruct-Turbo",
            "messages": [
                {"role": "system", "content": "You are a maths tutor."},
                {"role": "user", "content": "Explain the Pythagorean theorem."},
            ],
            "max_tokens": 1000,
        },
    }
    # Additional tasks can be added here
]

# Save tasks to a JSONL file (newline-delimited JSON)
file_name = "mybatchtest.jsonl"
with open(file_name, "w") as file:
    for task in tasks:
        file.write(json.dumps(task) + "\n")

## Uploading Batch job Files

This step uploads the input JSONL file to kluster.ai via the API. Once the file is uploaded, the API assigns a unique file ID. This ID is essential for subsequent steps, as it allows you to specify which file the batch job should use for processing

In [ ]:
batch_input_file = client.files.create(
    file=open(file_name, "rb"),
    purpose="batch"
)

batch_input_file.to_dict()

## Submit your Batch Job

This step starts your batch job by providing the uploaded file ID and setting the endpoint and completion window, initiating the batch process.

In [ ]:
batch_request = client.batches.create(
    input_file_id=batch_input_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

batch_request.to_dict()

## Monitor Job Progress

In this step, the batch job status is checked repeatedly to track its progress. You’ll see updates on the overall status and the number of completed tasks until the job is finished, failed, or cancelled.

In [ ]:
import time

# Poll the batch status until it's complete
while True:
    batch_status = client.batches.retrieve(batch_request.id)
    print("Batch status: {}".format(batch_status.status))
    print(
        f"Completed tasks: {batch_status.request_counts.completed} / {batch_status.request_counts.total}"
    )

    if batch_status.status.lower() in ["completed", "failed", "cancelled"]:
        break

    time.sleep(10)  # Wait for 10 seconds before checking again

batch_status.to_dict()

## Retrieve Results

In this step, the results of the batch job are retrieved if it completed successfully. The output is downloaded and saved to a local file for you to review. If the job failed, the status will indicate the issue.

In [ ]:
# Check if the batch completed successfully
if batch_status.status.lower() == "completed":
    # Retrieve the results
    result_file_id = batch_status.output_file_id
    results = client.files.content(result_file_id).content

    # Save results to a file
    result_file_name = "batch_results.jsonl"
    with open(result_file_name, "wb") as file:
        file.write(results)
    print(f"Results saved to {result_file_name}")
else:
    print(f"Batch failed with status: {batch_status.status}")

## List all Batch Jobs

This step lists the most recent batch jobs, providing an overview of their statuses and details.

In [ ]:
client.batches.list(limit=2).to_dict()

## Cancelling a Batch Job

To cancel a batch job that is currently in progress, invoke the batch cancel endpoint by providing the batch ID.

In [ ]:
client.batches.cancel(batch_request.id)

## List supported models

Find the right model for your job by first checking the list models endpoint. Choose from our range of models, optimized for different performance needs.

In [ ]:
client.models.list().to_dict()